In [ ]:
import sys
sys.path.append('..')

In [ ]:
from source.code.utils import preprocessing

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
ratings = pd.read_csv('data/BX-Book-Ratings.csv', sep=';', header=0, error_bad_lines=False, encoding='Windows-1251', low_memory=False)

In [ ]:
ratings.info()

In [ ]:
books = pd.read_csv('data/BX-Books.csv', sep=';', header=0, error_bad_lines=False, encoding='Windows-1251', low_memory=False)

In [ ]:
books.info()

In [ ]:
books['Year-Of-Publication'] = books['Year-Of-Publication'].astype(np.int)

In [ ]:
books.info()

In [ ]:
users = pd.read_csv('data/BX-Users.csv', sep=';', header=0, error_bad_lines=False, encoding='Windows-1251', low_memory=False)

In [ ]:
users.info()

# Preprocessing phase

In [ ]:
data_dict = {}
data_dict['books'] = books
data_dict['users'] = users
data_dict['ratings'] = ratings

In [ ]:
preprocessed_data_dict = preprocessing(data_dict=data_dict, is_explicit=True, book_ratings_count_threshold=2, user_ratings_count_threshold=2)

In [ ]:
preprocessed_data_dict['ratings'].head()

In [ ]:
preprocessed_data_dict['ratings'] = preprocessed_data_dict['ratings'].rename({'User-ID': 'userID', 'ISBN': 'itemID', 'Book-Rating': 'rating'}, axis='columns')

In [ ]:
preprocessed_data_dict['ratings'].head()

# Surprise experiments

In [ ]:
from surprise import SVD
from surprise import NMF
from surprise.prediction_algorithms.knns import KNNBasic
from surprise.model_selection import cross_validate
from surprise import Dataset
from surprise import Reader

## Data reader

In [ ]:
reader = Reader(rating_scale=(1, 5))

In [ ]:
data = Dataset.load_from_df(preprocessed_data_dict['ratings'][['userID', 'itemID', 'rating']], reader)

## SVD

In [ ]:
algo_svd = SVD()

In [ ]:
cross_validate(algo_svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True, n_jobs=-1)

## NMF

In [ ]:
algo_nmf = NMF()

In [ ]:
cross_validate(algo_nmf, data, measures=['RMSE', 'MAE'], cv=5, verbose=True, n_jobs=-1)

## KNN (Item-based)

In [ ]:
sim_options = {
    'name': 'cosine',
    'user_based': False
}

In [ ]:
algo_knn = KNNBasic(k=5, sim_options=sim_options)

In [ ]:
cross_validate(algo_knn, data, measures=['RMSE', 'MAE'], cv=5, verbose=True, n_jobs=1)

## KNN (User-based)

In [ ]:
sim_options = {
    'name': 'cosine'
}

In [ ]:
algo_knn = KNNBasic(k=5, sim_options=sim_options)

In [ ]:
cross_validate(algo_knn, data, measures=['RMSE', 'MAE'], cv=5, verbose=True, n_jobs=1)